# Ran in Google Colab

# Loading Text File from Kaggle into Dataframe

In [1]:
! pip install --ignore-installed pyspark==2.4.4

     |████████████████████████████████| 215.7 MB 59 kB/s 
     |████████████████████████████████| 197 kB 13.0 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130392 sha256=3b2deed8d172cc866866a27e6b5ff94276622a1b4fe643126bc8979ad2456a52
  Stored in directory: /root/.cache/pip/wheels/11/48/19/c3b6b66e4575c164407a83bc065179904ddc33c9d6500846f0
Successfully built pyspark


In [2]:
! pip install -q kaggle

In [30]:
from google.colab import files
files.upload()

{}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-03 07:25:55          17591  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          10495  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5172  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3795  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [7]:
! kaggle datasets download -d uciml/electric-power-consumption-data-set

 26% 5.00M/19.4M [00:00<00:00, 25.9MB/s]
100% 19.4M/19.4M [00:00<00:00, 64.7MB/s]


In [8]:
! unzip electric-power-consumption-data-set.zip

Archive:  electric-power-consumption-data-set.zip
  inflating: household_power_consumption.txt  


In [9]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.getOrCreate()

In [11]:
df = spark.read.text('household_power_consumption.txt')

In [12]:
df.show(5)

+--------------------+
|               value|
+--------------------+
|Date;Time;Global_...|
|16/12/2006;17:24:...|
|16/12/2006;17:25:...|
|16/12/2006;17:26:...|
|16/12/2006;17:27:...|
+--------------------+
only showing top 5 rows



In [13]:
dfColumnsNotSplit = df.selectExpr("split(value, ';')\
as Text_Data_In_A_Single_Column")

dfColumnsNotSplit.show(5,False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|Text_Data_In_A_Single_Column                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------+
|[Date, Time, Global_active_power, Global_reactive_power, Voltage, Global_intensity, Sub_metering_1, Sub_metering_2, Sub_metering_3]|
|[16/12/2006, 17:24:00, 4.216, 0.418, 234.840, 18.400, 0.000, 1.000, 17.000]                                                        |
|[16/12/2006, 17:25:00, 5.360, 0.436, 233.630, 23.000, 0.000, 1.000, 16.000]                                                        |
|[16/12/2006, 17:26:00, 5.374, 0.498, 233.290, 23.000, 0.000, 2.000, 17.000]                                                        |
|[16/12/2006, 17:27:00, 5.388, 0.502, 233.740, 23.000, 0.000, 

In [14]:
dfColumnsSplit = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv("household_power_consumption.txt")

dfColumnsSplit.show(5, truncate=False)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Date      |Time    |Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|4.216              |0.418                |234.840|18.400          |0.000         |1.000         |17.0          |
|16/12/2006|17:25:00|5.360              |0.436                |233.630|23.000          |0.000         |1.000         |16.0          |
|16/12/2006|17:26:00|5.374              |0.498                |233.290|23.000          |0.000         |2.000         |17.0          |
|16/12/2006|17:27:00|5.388              |0.502                |233.740|23.000          |0.000         |1.000         |17.0          |
|16/12/2006|17:28:00|3.666              |0.528                

# Select Query

## Show a single column

In [15]:
dfColumnsSplit.select("Global_active_power").show(5) 

+-------------------+
|Global_active_power|
+-------------------+
|              4.216|
|              5.360|
|              5.374|
|              5.388|
|              3.666|
+-------------------+
only showing top 5 rows



## Showing multiple columns

In [16]:
dfColumnsSplit.select("Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity").show(5)

+-------------------+---------------------+-------+----------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|
+-------------------+---------------------+-------+----------------+
|              4.216|                0.418|234.840|          18.400|
|              5.360|                0.436|233.630|          23.000|
|              5.374|                0.498|233.290|          23.000|
|              5.388|                0.502|233.740|          23.000|
|              3.666|                0.528|235.680|          15.800|
+-------------------+---------------------+-------+----------------+
only showing top 5 rows



# Like Query

In [17]:
dfColumnsSplit.select("Global_active_power", dfColumnsSplit.Global_active_power.like("5.388")).show(5)

+-------------------+------------------------------+
|Global_active_power|Global_active_power LIKE 5.388|
+-------------------+------------------------------+
|              4.216|                         false|
|              5.360|                         false|
|              5.374|                         false|
|              5.388|                          true|
|              3.666|                         false|
+-------------------+------------------------------+
only showing top 5 rows



# Filter Query

In [18]:
dfColumnsSplit.filter \
    (dfColumnsSplit.Global_active_power == "5.388").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|22/12/2006|20:53:00|              5.388|                0.252|232.360|          23.200|        36.000|         1.000|          17.0|
|24/12/2006|00:04:00|              5.388|                0.000|236.980|          22.600|         0.000|         0.000|          17.0|
|28/12/2006|19:00:00|              5.388|                0.176|234.950|          22.800|         0.000|        38.000|          17.0|
|28/12/2006|19:07:00|              5.388|                0.182

# Group By Query

In [19]:
dfColumnsSplit.groupBy("Date").count().show(5)

+----------+-----+
|      Date|count|
+----------+-----+
| 30/1/2007| 1440|
| 13/2/2007| 1440|
|19/11/2007| 1440|
| 12/1/2008| 1440|
| 26/2/2008| 1440|
+----------+-----+
only showing top 5 rows



# Handling NaNs 

## Finds NaNs

In [20]:
dfColumnsSplit.filter("Sub_metering_3 is NULL").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|21/12/2006|11:24:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|30/12/2006|10:08:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
|30/12/2006|10:09:00|                  ?|                    ?|      ?|               ?|             ?|             ?|          null|
| 14/1/2007|18:36:00|                  ?|                    ?

## Fill NaNs

In [21]:
dfColumnsNaNFill = dfColumnsSplit.na.fill(123456)

In [22]:
dfColumnsNaNFill.filter \
    (dfColumnsNaNFill.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|21/12/2006|11:24:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:08:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:09:00|                  ?|                    ?|      ?|               ?|             ?|             ?|      123456.0|
| 14/1/2007|18:36:00|                  ?|                    ?

# Modify Table

## Update Query - One Column's value


In [23]:
from pyspark.sql import functions as F

In [24]:
dfColumnsFill_OneColumn = dfColumnsNaNFill.withColumn \
    ("Global_active_power", F.when(F.col("Global_active_power")=='?', 100).otherwise(F.col("Global_active_power")))


In [25]:
dfColumnsFill_OneColumn.filter \
    (dfColumnsFill_OneColumn.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|21/12/2006|11:24:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:08:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
|30/12/2006|10:09:00|                100|                    ?|      ?|               ?|             ?|             ?|      123456.0|
| 14/1/2007|18:36:00|                100|                    ?

## Update Query - All Columns value


In [26]:
cols = dfColumnsNaNFill.columns
for col in cols:
  dfColumnsNaNFill = dfColumnsNaNFill.withColumn(col, F.when(dfColumnsNaNFill[col] == "?",100).otherwise(F.col(col)))

In [27]:
dfColumnsNaNFill.filter \
    (dfColumnsFill_OneColumn.Sub_metering_3 == "123456").show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2006|11:23:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|21/12/2006|11:24:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|30/12/2006|10:08:00|                100|                  100|    100|             100|           100|           100|      123456.0|
|30/12/2006|10:09:00|                100|                  100|    100|             100|           100|           100|      123456.0|
| 14/1/2007|18:36:00|                100|                  100

# Add Query

In [28]:
newRow = spark.createDataFrame([('21/12/2021','17:24:00',4.216,0.418,234.840,18.400,0.00,0.01,17.0)], dfColumnsSplit.columns)

dfColumnsSplit_NewRow = dfColumnsSplit.union(newRow)

In [29]:
dfColumnsSplit_NewRow.filter \
    (dfColumnsSplit_NewRow.Date == '21/12/2021').show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|21/12/2021|17:24:00|              4.216|                0.418| 234.84|            18.4|           0.0|          0.01|          17.0|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



In [31]:
spark.stop()